# Packages

In [ ]:
import qcodes as qc
from qcodes.logger.logger import start_all_logging
import datetime
import numpy as np
%matplotlib notebook
from datetime import date,datetime
import pyvisa
import sys
sys.path.append('') # Write the directory containing ZVL13_VNA.py
import json
from ZVL_VNA import ZVL

# Instrument and station initialization

In [ ]:
start_all_logging()


#Instruments
VNA = ZVL('ZVL', 'TCPIP0::vnarstafuri.fisica.unina.it::inst0::INSTR')


#Station inizialization
station = qc.Station()
station.add_component(VNA)

# Parameters definition

In [ ]:
#########################
##Parameters definition##
#########################

parameter_snap={}

S_parameter = 'S11'
meas_format = 'dbm'
start_freq = 4
end_freq = 8
#center_freq = 5
#span_freq = 1
VNA_power = -20
IF_bandwidth = 1000
Averages = 4

pnts = True  #Put True if you want to choose the number of points. 
pntsxGHz = False  #Put True if you want to choose the number of points per GHz.   
steppnts = False  #Put True if you want to choose the distance between the acquisition points in the frequency range of the VNA.

#Set VNA points
if pnts:    
    points_VNA = 101 #Max:4001
    VNA.npts(points_VNA)

#Set points per GHz
if pntsxGHz:    
    points_per_gigahz = 1000
    points_VNA = int((end_freq-start_freq)*points_per_gigahz+1)
    VNA.npts(points_VNA)

#Set distance between acquisition points (!!in Hz!!)
if steppnts:    
    step_freq = 1*1e3
    min_step_freq = ((end_freq - start_freq)*1e9)/4001
    max_step_freq = (end_freq - start_freq)*1e9
    if step_freq<min_step_freq:
        sys.exit("Step frequency must be higher than:" + str(round(min_step_freq*1e-3,1)) + " kHz")
    elif step_freq>max_step_freq:
        sys.exit("Step frequency must be lower than:" + str(round(max_step_freq*1e-6,1)) + " MHz")

VNA.calibration()
VNA.S_parameter(S_parameter)
VNA.format(meas_format)
VNA.bandwidth(IF_bandwidth)
VNA.power(VNA_power)
VNA.avg(Averages)
VNA.start(start_freq*1e9)
VNA.stop(end_freq*1e9)
#VNA.center(center_freq*1e9)
#VNA.span(span_freq*1e9)


# Trace acquisition

In [ ]:
#########################
##Parameters definition##
#########################

S_parameter = 'S11'
meas_format = 'dbm'
start_freq = 4 #GHz
end_freq = 8 #GHz
#center_freq = 5 #GHz
#span_freq = 1 #GHz
VNA_power = -20 #dBm
IF_bandwidth = 1000 #Hz
Averages = 4
points_VNA = 101 #Max:4001


#Parameters setting in the VNA
VNA.calibration() #Inserts the calibration
VNA.S_parameter(S_parameter)
VNA.npts(points_VNA)
VNA.format(meas_format)
VNA.bandwidth(IF_bandwidth)
VNA.power(VNA_power)
VNA.avg(Averages)
VNA.start(start_freq*1e9)
VNA.stop(end_freq*1e9)
#VNA.center(center_freq*1e9)
#VNA.span(span_freq*1e9)

#############################
##Experiment initialization##
#############################

meas_type = 'trace_acq'

#Experiment definition
exp_name = 'exp_name'
sample = 'sample'
exp=qc.load_or_create_experiment(experiment_name=exp_name,
                          sample_name=sample)



#Parameters registration
meas = qc.Measurement(exp=exp, station=station) 
meas.register_parameter(VNA.trace) 
meas.register_parameter(VNA.S_trace)

#Start measurement
VNA.rf_power(1) #Switch on the power
VNA.cont_meas_on() #Switch on the sweep
VNA.electrical_delay_auto() #Sets the electrical delay automatically
VNA.autoscale() 

with meas.run() as datasaver:
    get_v = VNA.trace.get()
    get_v2 = VNA.S_trace.get()
    datasaver.add_result((VNA.trace, get_v))
    datasaver.add_result((VNA.S_trace, get_v2))


#Shutdown devices
VNA.rf_power(0)
VNA.cont_meas_off()

###############
##Data saving##
###############

captured_run_id=datasaver.dataset.run_id
dataset = qc.load_by_run_spec(captured_run_id=captured_run_id)
freq = dataset.get_parameter_data('ZVL_ZVL_frequency')['ZVL_ZVL_frequency']['ZVL_ZVL_frequency']
save_trace = dataset.get_parameter_data('ZVL_trace')['ZVL_trace']['ZVL_trace']
S_trace = dataset.get_parameter_data('ZVL_S_trace')['ZVL_S_trace']['ZVL_S_trace']
params = json.loads(dataset.metadata['parameter_snap'])